### Mean model with lags = [1,5,22] using pct_high gives lower log likelihood than GJR with pct_close

In [98]:
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://localhost',
                       connect_args={'read_default_file': '~/.mytest.cnf'})
conn = engine.connect()

In [99]:
import pandas as pd

tck_qry = pd.read_sql_query('''select distinct us.symbol, avg(us.volume*us.close) from USE_2021 us
join US_SymsWithOptions USWO on us.symbol = USWO.Symbol
where USWO.`Symbol Type` = 'Equity'
    group by us.symbol order by avg(us.volume*us.close) desc limit 50''',
                            conn)
tdf = pd.DataFrame(tck_qry, columns=['symbol'])
tickers = tdf['symbol']
prices = ["pct_open", "pct_high", "pct_low", "pct_close"]
# prices = ["pct_open", "pct_high", "pct_low", "pct_close",
#           'pct_hilo', 'pct_opclo']

In [100]:
def query_symbol(sym, conn):
    qry = pd.read_sql_query(
        '''select tradedate, pct_open, pct_high, pct_low, pct_close,
         pct_hilo, pct_opclo from USEQ_HIST
        where symbol="''' + sym + '''"
        and volume>0
         order by tradedate''',
        conn,
        index_col='tradedate'
    )
    return sym, pd.DataFrame(qry, columns=prices)

In [101]:
import math
def ann_var(var):
    return (math.sqrt((var)*252))


In [102]:
from arch.univariate import ARX, GARCH, StudentsT

from arch import arch_model
def gjr(df):
    return "gjr", arch_model(df, rescale=False, p=1, o=1, q=1, dist="StudentsT")
def arx(df):
    am=ARX(df, rescale=False, lags=[1,5,22], constant=True)
    am.volatility = GARCH(1,1,1)
    am.distribution = StudentsT()
    return "arx", am



In [103]:
rows = []

for tk in tickers:
    ticker, dft = query_symbol(tk, conn)
    for pr in prices:
        if(len(dft.index)<800):
            continue
        df = dft[pr]
        for str, am in (gjr(df), arx(df)):
            res = am.fit(disp='off')
            if res.convergence_flag != 0:
                continue
            forecasts = res.forecast(reindex=False)
            lhood = res.loglikelihood
            if str == 'arx':
                nnobs = res.nobs
                lhood = lhood*(nnobs/(nnobs - 22))
            row = [tk, str, pr, lhood, ann_var(forecasts.variance['h.1'].iloc[0])
               ,forecasts.mean['h.1'].iloc[0], res.nobs]
            rows.append(row)
    cmp = pd.DataFrame(rows, columns=['ticker', 'model', 'price', 'lhood', 'volatilty', 'mean', 'nobs'])

compare = cmp.set_index(['ticker', 'model', 'price'])
compare.head(40)



lhood  volatilty      mean  nobs
ticker model price                                            
TSLA   gjr   pct_open  -3743.207729  47.734052  0.161600  1468
       arx   pct_open  -3744.960593  48.101799  0.180110  1446
       gjr   pct_high  -3591.419980  71.636216  0.168396  1468
       arx   pct_high  -3580.401205  67.559279  0.887676  1446
       gjr   pct_low   -3637.192561  42.350703  0.237877  1468
       arx   pct_low   -3619.478782  39.330864  0.466653  1446
       gjr   pct_close -3709.211032  50.450455  0.190670  1468
       arx   pct_close -3707.187974  51.618713  0.034125  1446
AAPL   gjr   pct_open  -2700.213162  21.779809  0.162901  1468
       arx   pct_open  -2686.278407  21.901612  0.257523  1446
       gjr   pct_high  -2484.354531  23.924716  0.137673  1468
       arx   pct_high  -2467.223190  24.904504 -0.091659  1446
       gjr   pct_low   -2619.972295  25.384211  0.179015  1468
       arx   pct_low   -2604.626199  25.942039 -0.012638  1446
       gjr   pct_close -2693.329445  23.236471  0.145183  1468
       arx   pct_close -2678.547825  22.896801  0.208354  1446
AMZN   gjr   pct_open  -2788.366045  33.516284  0.151898  1468
       arx   pct_open  -2775.596558  33.365221  0.271391  1446
       gjr   pct_high  -2560.122360  31.860321  0.137003  1468
       arx   pct_high  -2525.908450  33.105431 -0.375041  1446
       gjr   pct_low   -2717.380566  30.737718  0.145348  1468
       arx   pct_low   -2694.805536  31.274647 -0.135335  1446
       gjr   pct_close -2736.398015  28.854838  0.138219  1468
       arx   pct_close -2715.748619  29.035857  0.098887  1446
MSFT   gjr   pct_open  -2468.375727  16.420729  0.125827  1468
       arx   pct_open  -2454.756370  16.398028  0.152767  1446
       gjr   pct_high  -2231.751939  17.799973  0.143952  1468
       arx   pct_high  -2219.732981  18.084902  0.243105  1446
       gjr   pct_low   -2388.346367  14.970530  0.126529  1468
       arx   pct_low   -2371.967765  14.894590  0.171411  1446
       gjr   pct_close -2508.753730  19.410783  0.126804  1468
       arx   pct_close -2485.190164  20.532631 -0.048478  1446
FB     gjr   pct_open  -2874.550425  36.601126  0.117026  1468
       arx   pct_open  -2857.909117  37.220473 -0.028625  1446
       gjr   pct_high  -2633.329773  31.072540  0.102749  1468
       arx   pct_high  -2615.371423  30.136992  0.118690  1446
       gjr   pct_low   -2752.865517  38.754396  0.142233  1468
       arx   pct_low   -2730.063040  39.229451  0.353165  1446
       gjr   pct_close -2837.396062  40.486181  0.114709  1468
       arx   pct_close -2817.572212  39.613023 -0.045115  1446